## Enviorment

In [1]:
!whoami

michael_zl_prime


In [2]:
!gcloud auth list

                  Credentialed Accounts
ACTIVE  ACCOUNT
*       312164605303-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [3]:
#test which virtualenv running in
import sys
sys.prefix

'/usr'

---

In [4]:
import ipywidgets as widgets

In [5]:
import tensorflow as tf
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from google.cloud import storage

from config import TRANSFORMED_TRAIN_ANNOTATIONS_PATH,TRANSFORMED_VALIDATION_ANNOTATIONS_PATH,IMAGE_SIZE
from models.six_stage_linear_model import ModelMaker
import dataset_functions
import visualizations as v

In [6]:
tf.version.VERSION

'2.1.0-dev20191124'

In [7]:
#Training config, can be moved later to main config
CACHE=False
CACHE_RAMFS=False #uses a ramfs file to force using main memory
BATCH_SIZE=32  #must be small if caching
SHUFFLE=True
PREFETCH=10  #size of prefetch size, 0 to disable

In [8]:
tpu_ip='10.0.3.2'
# !!!MAKE SURE THE TPU ADDRESS IS CORRECT!!
# 1.ip must be correct
# 2.tpu must be turned on!
# 3.version must be 'nightly-2.x'
# 4.tpu must be reachable (check with gce netowrking/connectivity test)
# if not this will hang
tpu_address = 'grpc://'+tpu_ip+':8470'
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: 10.0.3.2:8470


INFO:tensorflow:Initializing the TPU system: 10.0.3.2:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


---
Caching using ramfs (irrelevant for tpu)

In [9]:
if CACHE_RAMFS:
    !mkdir /tmp/ramdisk
    !sudo umount /tmp/ramdisk
    !sudo mount -t ramfs -o size=512m ramfs /tmp/ramdisk
    !sudo chown $LOGNAME:$LOGNAME /tmp/ramdisk

In [10]:
if CACHE_RAMFS:
    cache_loc="/tmp/ramdisk/cache_t"
    cache_v_loc="/tmp/ramdisk/cache_v"
else:
    cache_loc=None
    cache_v_loc=None

In [11]:
cache_loc

---
# Make dataset

In [12]:
label_transformer=dataset_functions.LabelTransformer()
@tf.function
def make_label_tensors(elem):
    """Transforms a dict data element:
    1.Read jpg to tensor 
    1.1 Resize img to correct size for network
    2.Convert keypoints to correct form label tensor
    3.Convert joints to correct form label tensor
    outputs a tuple data element"""
    
    idd=elem['id']
    kpt_tr=label_transformer.keypoints_spots_vmapfn(elem['kpts'])
    paf_tr=label_transformer.joints_PAFs(elem['joints'])
    
    image_raw=elem["image_raw"]
    image=tf.image.decode_jpeg(image_raw,channels=3)
    image=tf.image.convert_image_dtype(image,dtype=tf.float32)
    image=tf.image.resize(image,IMAGE_SIZE)
    return image,(paf_tr,kpt_tr),idd

In [13]:
@tf.function
def place_training_labels(image,labels,idd):
    """Disterbutes labels into the correct configuration for the model, ie 4 PAF stage, 2 kpt stages
    must match the model"""
    paf_tr=labels[0]
    kpt_tr=labels[1]
    return image,(paf_tr,paf_tr,paf_tr,paf_tr,kpt_tr,kpt_tr) #this should match the model outputs, and is different for each model

Read and Parse the TFrecords

In [14]:
DATASET_SIZE=56000 #exact size not critical
DATASET_VAL_SIZE=2500 

----
figure out GCS storage


In [15]:
train_prefix=TRANSFORMED_TRAIN_ANNOTATIONS_PATH.split(os.sep)[-1]
val_prefix=TRANSFORMED_VALIDATION_ANNOTATIONS_PATH.split(os.sep)[-1]

In [16]:
bucket_name="datasets_bucket_a"
gs_prefix="gs://"

In [17]:
storage_client = storage.Client() #must have apropriate authenitication to work 

train_blobs = storage_client.list_blobs(bucket_name,prefix=train_prefix)
val_blobs = storage_client.list_blobs(bucket_name,prefix=val_prefix)

In [18]:
tfrecord_files_train=[gs_prefix+bucket_name+'/'+blob.name for blob in train_blobs]
tfrecord_files_val=[gs_prefix+bucket_name+'/'+blob.name for blob in val_blobs]

In [19]:
#tfrecord_files_train

In [20]:
ds = tf.data.TFRecordDataset(tfrecord_files_train)

In [21]:
TF_parser=dataset_functions.TFrecordParser() #used for 

#order of transformations is critical!

#TFrecord files to raw format
ds = tf.data.TFRecordDataset(tfrecord_files_train) #numf reads can be put here, but I don't think I/O is the bottleneck

#raw format to imgs,tensors(coords kpts)
ds=ds.map(TF_parser.read_tfrecord)

#cache  ,caching is here before decompressing jpgs and label tensors (should be ~9GB) , (full dataset should be ~90, cache later if RAM aviable)
if CACHE: ds=ds.cache(cache_loc)
if SHUFFLE: ds=ds.shuffle(100)    
    
#Augmentation should be here, to operate on smaller tensors
    
#imgs,tensors to label_tensors (46,46,17/38)
ds=ds.map(make_label_tensors)
#imgs,label_tensors arrange for model outputs
ds=ds.map(place_training_labels) 

#batch
ds=ds.batch(BATCH_SIZE)
#repeat
ds=ds.repeat()
#prefetch
if PREFETCH: ds=ds.prefetch(PREFETCH)

Make validation dataset

In [22]:
TF_parser=dataset_functions.TFrecordParser() #used for 

#order of transformations is critical!

#TFrecord files to raw format
ds_v = tf.data.TFRecordDataset(tfrecord_files_val) #numf reads can be put here, but I don't think I/O is the bottleneck
#raw format to imgs,tensors(coords kpts)
ds_v=ds_v.map(TF_parser.read_tfrecord)   

#cache  
if CACHE: ds_v=ds_v.cache(cache_v_loc)
    
#imgs,tensors to label_tensors (46,46,17/38)
ds_v=ds_v.map(make_label_tensors)
#imgs,label_tensors arrange for model outputs
ds_v=ds_v.map(place_training_labels) 
#batch
ds_v=ds_v.batch(BATCH_SIZE)


Examine datasets

In [23]:
st=next(iter(ds))
#st
#st_v=next(iter(ds_v))
#v.show_pafs_kpts_img()

In [24]:
len(st[0])

32

In [25]:
ds

<PrefetchDataset shapes: ((None, 368, 368, 3), ((None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 17), (None, 46, 46, 17))), types: (tf.float32, (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>

In [26]:
ds_v

<BatchDataset shapes: ((None, 368, 368, 3), ((None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 17), (None, 46, 46, 17))), types: (tf.float32, (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>

In [27]:
#st[0]

---
# Callbacks
**Checkpoints**

In [48]:
#local dir for CPU/GPU training
#checkpoints_dir='./checkpoints'
#!mkdir checkpoints
#gcs dir for TPU training
checkpoints_dir='gs://dl_training_results/checkpoints'
#make sure the directory exists, if not make it.

In [29]:
import datetime
t=datetime.datetime.now()
now=t.strftime("%a%d%m%y-%H%M")
now

'Wed271119-1313'

In [30]:
checkpoints_path = checkpoints_dir+"/ModelWeights-"+now+"-{epoch:04d}.ckpt" 
checkpoints_path

'gs://dl_training_results/checkpoints/ModelWeights-Wed271119-1313-{epoch:04d}.ckpt'

In [31]:
#if this fails, the checkpointing won't work
!touch /tmp/test
!gsutil cp /tmp/test {checkpoints_dir}/test
!gsutil rm {checkpoints_dir}/test

Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/checkpoints/test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


In [32]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoints_path,
                                             save_weights_only=True,
                                             verbose=1)

**Logging**

In [65]:
logging_dir='gs://dl_training_results/logging/'
csv_logger = tf.keras.callbacks.CSVLogger(logging_dir)


In [66]:
#if this fails, the checkpointing won't work
!touch /tmp/test
!gsutil cp /tmp/test {logging_dir}/test
!gsutil rm {logging_dir}/test

Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/logging//test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


**TensorBoard**

In [67]:
tensorboard_dir='gs://dl_training_results/tensorboard/'
tensorboard_callback=tf.keras.callbacks.TensorBoard(
    log_dir=tensorboard_dir
    ,update_freq=5000 #to update sooner than every epoch
)

In [63]:
#if this fails, the checkpointing won't work
!touch /tmp/test
!gsutil cp /tmp/test {tensorboard_dir}/test
!gsutil rm {tensorboard_dir}/test

Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/tensorboard/test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


In [35]:
#the command to run tensorboard
#tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all

---
# Model

In [36]:
@tf.function
def mse_2d_loss(y_true, y_pred):
    pixel_losses=tf.keras.losses.mean_squared_error(y_true, y_pred)
    return tf.math.reduce_mean(pixel_losses,axis=-1)

In [76]:
class AnalogRecallError(tf.keras.metrics.Metric):
    """This metric returns the overlap error of the true gaussian 'islands' and the predicted ones"""
    def __init__(self, name='analog_recall_error',thershold=0.01, **kwargs):
        super(AnalogRecallError, self).__init__(name=name, **kwargs)
        self.mean = self.add_weight(name='mean', initializer='zeros')
        self.thershold=thershold
    
    def update_state(self, y_true, y_pred,**kwargs):        
        true_island_sum=tf.reduce_sum(tf.where(y_true>self.thershold,y_true,0)) #get sum of the true island
        true_island_size=tf.cast(tf.math.count_nonzero(y_true>self.thershold),dtype=tf.float32) #get size of the island   
        mean_island_true=true_island_sum/true_island_size #average island value
               
        err=y_true-y_pred  #get all error
        recall_err=tf.where(err>0,err,0)  #get only recall error, the parts where prediction is missing
        recall_err_sum=tf.reduce_sum(recall_err)        
        err_island_size=tf.cast(tf.math.count_nonzero(y_true>self.thershold),tf.float32)  #get size of the islands above thershold
        
        mean_island_recall_err=recall_err_sum/err_island_size  #mean of the error 
        
        value=mean_island_recall_err/mean_island_true
        self.mean.assign_add(value)
    def result(self):
        return self.mean
class AnalogRecall(tf.keras.metrics.Metric):
    """This metric returns the overlap of the true gaussian 'islands' and the predicted ones"""
    def __init__(self, name='analog_recall_error',thershold=0.01, **kwargs):
        super(AnalogRecall, self).__init__(name=name, **kwargs)
        self.mean = self.add_weight(name='mean', initializer='zeros')
        self.thershold=thershold
    
    @tf.function
    def update_state(self, y_true, y_pred,**kwargs):     
        
        true_island_sum=tf.reduce_sum(tf.where(y_true>self.thershold,y_true,0)) #get sum of the true island
        true_island_size=tf.cast(tf.math.count_nonzero(y_true>self.thershold),dtype=tf.float32) #get size of the island   
        mean_island_true=true_island_sum/true_island_size #average island value
               
        err=y_true-y_pred  #get all error
        recall_err=tf.where(err>0,err,0)  #get only recall error, the parts where prediction is missing
        recall_err_sum=tf.reduce_sum(recall_err)        
        err_island_size=tf.cast(tf.math.count_nonzero(y_true>self.thershold),tf.float32)  #get size of the islands above thershold
        
        mean_island_recall_err=recall_err_sum/err_island_size  #mean of the error 
        
        value=1-mean_island_recall_err/mean_island_true #the 1- converts it to recall accuracy onstead of err        
        self.mean.assign_add(value)
    def result(self):
        return self.mean

In [77]:
with strategy.scope():
    model_maker=ModelMaker()
    train_model,test_model=model_maker.create_models()
    
    #this must match the model output order
    metrics={'stage1paf_output':  [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
         ,'stage2paf_output':  [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
         ,'stage3paf_output':  [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
         ,'stage4paf_output':  [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
         ,'stage5heatmap_output': [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]    
         ,'stage6heatmap_output': [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
        }
    
    train_model.compile(optimizer=tf.keras.optimizers.Adam()
                    ,loss=mse_2d_loss
                    ,metrics=metrics                    
                   )


TypeError: To be compatible with tf.contrib.eager.defun, Python functions must return zero or more Tensors; in compilation of <function AnalogRecall.update_state at 0x7ff2121311e0>, found return value of type <class 'tensorflow.python.framework.ops.Operation'>, which is not a Tensor.

## Load weights

In [71]:
#load latest weights
latest = tf.train.latest_checkpoint(checkpoints_dir)
if latest:
    print("Found checkpoint: %s" % latest)

Found checkpoint: gs://dl_training_results/checkpoints/ModelWeights-Wed271119-1313-0001.ckpt


In [72]:
load=widgets.Checkbox(
    value=False,
    description='Load last weights'
)
load

Checkbox(value=False, description='Load last weights')

In [73]:
if latest and (load.value):
    train_model.load_weights(latest)

---
Actually training

In [74]:
steps_per_epoch=int(DATASET_SIZE/BATCH_SIZE)

In [75]:
train_model.fit(
    ds,epochs=2
    ,steps_per_epoch=7
    ,validation_data=ds_v
    ,callbacks=[cp_callback
                #,csv_logger
                ,tensorboard_callback]
)

Train for 7 steps
Epoch 1/2
1/7 [===>..........................] - ETA: 6:02 - loss: 0.3440 - stage1paf_output_loss: 0.0464 - stage2paf_output_loss: 0.0478 - stage3paf_output_loss: 0.0755 - stage4paf_output_loss: 0.0272 - stage5heatmap_output_loss: 0.0668 - stage6heatmap_output_loss: 0.0804 - stage1paf_output_analog_recall_error: 1.6352 - stage1paf_output_mean_absolute_error: 0.1167 - stage2paf_output_analog_recall_error: 2.3400 - stage2paf_output_mean_absolute_error: 0.1185 - stage3paf_output_analog_recall_error: 1.8509 - stage3paf_output_mean_absolute_error: 0.1365 - stage4paf_output_analog_recall_error: 1.7809 - stage4paf_output_mean_absolute_error: 0.0875 - stage5heatmap_output_analog_recall_error: 1.6615 - stage5heatmap_output_mean_absolute_error: 0.1488 - stage6heatmap_output_analog_recall_error: 2.5395 - stage6heatmap_output_mean_absolute_error: 0.1835WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.064939). Check your callbacks.


2/7 [=======>......................] - ETA: 2:35 - loss: 0.3188 - stage1paf_output_loss: 0.0265 - stage2paf_output_loss: 0.0472 - stage3paf_output_loss: 0.0567 - stage4paf_output_loss: 0.0462 - stage5heatmap_output_loss: 0.0663 - stage6heatmap_output_loss: 0.0759 - stage1paf_output_analog_recall_error: 3.1359 - stage1paf_output_mean_absolute_error: 0.0760 - stage2paf_output_analog_recall_error: 3.1956 - stage2paf_output_mean_absolute_error: 0.1034 - stage3paf_output_analog_recall_error: 2.9142 - stage3paf_output_mean_absolute_error: 0.1090 - stage4paf_output_analog_recall_error: 3.7588 - stage4paf_output_mean_absolute_error: 0.0931 - stage5heatmap_output_analog_recall_error: 3.0868 - stage5heatmap_output_mean_absolute_error: 0.1433 - stage6heatmap_output_analog_recall_error: 3.3609 - stage6heatmap_output_mean_absolute_error: 0.1528WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.603885). Check your callbacks.


3/7 [===========>..................] - ETA: 1:23 - loss: 0.2458 - stage1paf_output_loss: 0.0180 - stage2paf_output_loss: 0.0318 - stage3paf_output_loss: 0.0382 - stage4paf_output_loss: 0.0312 - stage5heatmap_output_loss: 0.0596 - stage6heatmap_output_loss: 0.0669 - stage1paf_output_analog_recall_error: 3.4472 - stage1paf_output_mean_absolute_error: 0.0524 - stage2paf_output_analog_recall_error: 3.2847 - stage2paf_output_mean_absolute_error: 0.0706 - stage3paf_output_analog_recall_error: 2.9614 - stage3paf_output_mean_absolute_error: 0.0741 - stage4paf_output_analog_recall_error: 3.9114 - stage4paf_output_mean_absolute_error: 0.0641 - stage5heatmap_output_analog_recall_error: 3.0712 - stage5heatmap_output_mean_absolute_error: 0.1238 - stage6heatmap_output_analog_recall_error: 4.0590 - stage6heatmap_output_mean_absolute_error: 0.1357WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.329276). Check your callbacks.


6/7 [========================>.....] - ETA: 10s - loss: 0.1649 - stage1paf_output_loss: 0.0094 - stage2paf_output_loss: 0.0164 - stage3paf_output_loss: 0.0195 - stage4paf_output_loss: 0.0160 - stage5heatmap_output_loss: 0.0499 - stage6heatmap_output_loss: 0.0536 - stage1paf_output_analog_recall_error: 3.6019 - stage1paf_output_mean_absolute_error: 0.0273 - stage2paf_output_analog_recall_error: 3.3257 - stage2paf_output_mean_absolute_error: 0.0367 - stage3paf_output_analog_recall_error: 2.9743 - stage3paf_output_mean_absolute_error: 0.0381 - stage4paf_output_analog_recall_error: 4.2060 - stage4paf_output_mean_absolute_error: 0.0339 - stage5heatmap_output_analog_recall_error: 3.5074 - stage5heatmap_output_mean_absolute_error: 0.0997 - stage6heatmap_output_analog_recall_error: 4.6416 - stage6heatmap_output_mean_absolute_error: 0.1067WARNING:tensorflow:5 out of the last 13 calls to <function mse_2d_loss at 0x7ff2c092a620> triggered tf.function retracing. Tracing is expensive and the excess


Epoch 00001: saving model to gs://dl_training_results/checkpoints/ModelWeights-Wed271119-1313-0001.ckpt
7/7 [==============================] - 137s 20s/step - loss: 0.1518 - stage1paf_output_loss: 0.0082 - stage2paf_output_loss: 0.0141 - stage3paf_output_loss: 0.0169 - stage4paf_output_loss: 0.0139 - stage5heatmap_output_loss: 0.0478 - stage6heatmap_output_loss: 0.0509 - stage1paf_output_analog_recall_error: 3.5961 - stage1paf_output_mean_absolute_error: 0.0236 - stage2paf_output_analog_recall_error: 3.3317 - stage2paf_output_mean_absolute_error: 0.0317 - stage3paf_output_analog_recall_error: 2.9671 - stage3paf_output_mean_absolute_error: 0.0329 - stage4paf_output_analog_recall_error: 4.2744 - stage4paf_output_mean_absolute_error: 0.0294 - stage5heatmap_output_analog_recall_error: 3.9228 - stage5heatmap_output_mean_absolute_error: 0.0961 - stage6heatmap_output_analog_recall_error: 5.0445 - stage6heatmap_output_mean_absolute_error: 0.1022 - val_loss: 0.0845 - val_stage1paf_output_loss:

6/7 [========================>.....] - ETA: 0s - loss: 0.0920 - stage1paf_output_loss: 6.8587e-04 - stage2paf_output_loss: 6.8590e-04 - stage3paf_output_loss: 6.8548e-04 - stage4paf_output_loss: 6.8668e-04 - stage5heatmap_output_loss: 0.0442 - stage6heatmap_output_loss: 0.0450 - stage1paf_output_analog_recall_error: -0.0689 - stage1paf_output_mean_absolute_error: 0.0013 - stage2paf_output_analog_recall_error: -0.0561 - stage2paf_output_mean_absolute_error: 0.0013 - stage3paf_output_analog_recall_error: -0.0691 - stage3paf_output_mean_absolute_error: 0.0012 - stage4paf_output_analog_recall_error: 0.0374 - stage4paf_output_mean_absolute_error: 0.0015 - stage5heatmap_output_analog_recall_error: 2.0278 - stage5heatmap_output_mean_absolute_error: 0.0882 - stage6heatmap_output_analog_recall_error: 1.8380 - stage6heatmap_output_mean_absolute_error: 0.0893
Epoch 00002: saving model to gs://dl_training_results/checkpoints/ModelWeights-Wed271119-1313-0002.ckpt
7/7 [==============================

In [ ]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data

# !!!!!!!IT IS FUCKING WORKING!!!!!!!